In [ ]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout

### This is a tutorial for creating a treatment train one unit process at a time. The aim is to demonstrate design capabilities in WaterTAP3 using the "treatment_train_design" module. The steps are:
#### 1. Look up unit process library
#### 2. Look up source and end use source water options
#### 3. Create a treatment train with a unit process, defining the source and end use
#### 4. Add another unit process in parallel or series
#### 5. Add multiple water sources
#### 6. Add multiple end use water source
#### 7. Add a stream from any point to any point
#### 8. Remove unit process
#### 9. Save and load treatment train

##### Import WaterTAP3 Package

In [ ]:
import watertap as wt

##### Step 1: Look up unit process library. Returns a List.

In [ ]:
wt.unit_process_library_list

##### Step 2: Look up source and end use source water options. Returns a Pandas dataframe.

In [ ]:
# source node name, feedwater column not used. TODO.
wt.water_source_use_library.head()

##### Step 3: Create a treatment train with a unit process, defining the source and end use

In [ ]:
# unit_process_name_list needs to be a list
unit_process_name_list = ['microfiltration_twb']
T = wt.treatment_train_design.create_train(unit_process_name_list=unit_process_name_list,
                                            source_water_reference = 'Poseidon',
                                            source_water_type = 'Wastewater',
                                            source_water_case_study ='Typical untreated domestic wastewater',
                                            enduse_water_reference = 'Poseidon',
                                            enduse_water_type = 'Environmental',
                                            enduse_water_case_study = 'EPA: Environmental Reuse',
                                            flow = 0)

In [ ]:
# display command
wt.display.show_train2(T)

##### Step 4: Add another unit process in parallel or series

In [ ]:
##### ADD UNIT PROCESS IN PARALLEL ####
T = wt.treatment_train_design.add_unit_process(T, from_node = 'source',
                                                to_node = 'use', 
                                                unit_process_name_list = ['ultrafiltration_twb'],
                                                unit_process_list = ['ultrafiltration_twb'],
                                                order = 'parallel')

In [ ]:
wt.display.show_train2(T)

In [ ]:
##### ADD UNIT PROCESS IN SERIES ####
T = wt.treatment_train_design.add_unit_process(T, from_node = 'use',
                                                to_node = None, 
                                                unit_process_name_list = ['chlorination_twb'],
                                                unit_process_list = ['chlorination_twb'],
                                                order = 'series')

In [ ]:
wt.display.show_train(T)

##### Step 5: Add multiple water sources

In [ ]:
# ADD ANOTHER SOURCE. NEED TO PROVIDE A NEW SOURCE NAME.
source_to_node_names = [['source1', 'ultrafiltration_twb_start']]
T = wt.treatment_train_design.add_multiple_water_sources(T,
                                              number_of_additional_sources = 1,
                                              source_water_reference = 'Poseidon',
                                              source_water_type = 'Wastewater',
                                              source_water_case_study = 'Typical untreated domestic wastewater',
                                              source_to_node_names=source_to_node_names,
                                              flow = 0)

In [ ]:
wt.display.show_train(T)

##### Step 6: Add multiple end use water source

In [ ]:
# ADD ANOTHER END USE 
node_to_use_names = [['chlorination_twb_end', 'use2']]
T = wt.treatment_train_design.add_multiple_water_enduses(T,
                                              number_of_additional_ends = 1,
                                              enduse_water_reference = 'Poseidon',
                                              enduse_water_type = 'Wastewater',
                                              enduse_water_case_study = 'Typical untreated domestic wastewater',
                                              node_to_end_names= node_to_use_names,
                                              min_flow = 0)

In [ ]:
wt.display.show_train(T)

##### Step 7: Add a stream from any point to any point. For this, user must define the type of stream being added. Options are: feedwater_stream, waste_stream, use_stream, recycle_stream, source_and_recycle_stream, treatment_process

In [ ]:
for edge in T.edges():
    print(T.edges[edge]['name'], '----->>>' , T.edges[edge]['type'])

In [ ]:
T = wt.treatment_train_design.add_stream(T, 
                                         from_node = 'microfiltration_twb_end', 
                                        to_node = 'use2',
                                        stream_name = 'treated_stream_example',
                                        stream_type = 'treated_stream') # TODO

In [ ]:
wt.display.show_train(T)

##### Step 8: Remove unit process

In [ ]:
T = wt.filter_processes.remove_single_process(T, process_name = 'ultrafiltration_twb')

In [ ]:
wt.display.show_train(T)

##### Step 9: Save and load treatment train

In [ ]:
#### SAVE TRAIN ####
path = 'trains/Tutorial1_treatment_train_example.csv'
wt.save_train(T, path)

In [ ]:
#### LOAD TRAIN ####
path = 'trains/Tutorial1_treatment_train_example.csv'
TT = wt.load_train(path)

In [ ]:
wt.display.show_train(TT)